In [ ]:
# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F # FFFFF

# Data loading
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

# Auxiliary functions
from torch.utils.tensorboard import SummaryWriter  # Used for Tensorboard logging
import os
import numpy as np
import matplotlib.pyplot as plt
from math import floor, ceil
import datetime

# Segmentación de imagen mediante arquitectura U-Net

In [ ]:
PATH_ROOT = os.path.join('.')
# Ruta para datos:
PATH_DATA = os.path.join(PATH_ROOT, 'data')
# Ruta para modelos:
PATH_MODELS = os.path.join(PATH_ROOT, 'reports', 'models')
# Ruta para resultados:
PATH_RESULTS = os.path.join(PATH_ROOT, 'reports', 'results')
# Ruta para ejecuciones:
PATH_RUNS = os.path.join(PATH_ROOT, 'reports', 'runs')

In [ ]:
# Para cada sesión creamos un directorio nuevo, a partir de la fecha y hora de su ejecución:
date = datetime.datetime.now()
test_name = str(date.year) + '_' + str(date.month) + '_' +  str(date.day) + '__' + str(date.hour) + '_' + str(date.minute)
print('Nombre del directorio de pruebas: {}'.format(test_name))
models_folder = os.path.join(PATH_MODELS, test_name)
try:
    os.makedirs(models_folder)
except:
    print(f'Folder {models_folder} already existed.')
results_folder = os.path.join(PATH_RESULTS, test_name)
try:
    os.makedirs(results_folder)
except:
    print(f'Folder {results_folder} already existed.')
runs_folder = os.path.join(PATH_RUNS, test_name)
try:
    os.makedirs(runs_folder)
except:
    print(f'Folder {runs_folder} already existed.')


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

## Definición del modelo

Como vamos a repetir la estructura de Convolución, Convolución, pooling, hacemos una clase para ello

In [ ]:
class dobleConvolucionMaxPool(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.convolucion_1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)  
        self.convolucion_2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    
    def forward(self, x):
        x = torch.nn.functional.relu(self.convolucion_1(x))
        x = torch.nn.functional.relu(self.convolucion_2(x))
        x = self.pool(x)
        return x

Lo mismo para hacer las deconvoluciones y el unpooling

In [ ]:
class dobleDeconvolucionMaxUnpool(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.convolucion_1 = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=3, padding=1)  
        self.convolucion_2 = nn.ConvTranspose2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.unpool = nn.MaxUnpool2d(kernel_size=2, stride=2)
    
    def forward(self, x):
        x = torch.nn.functional.relu(self.convolucion_1(x))
        x = torch.nn.functional.relu(self.convolucion_2(x))
        x = self.pool(x)
        return x